In [5]:
import sys
sys.path.insert(0,'/home/zahradnik/PycharmProjects/master_thesis')

In [6]:
import numpy as np
import pandas as pd
import os
from src.modules import conf, fit, df_columns, evl, preprocess
from sklearn.model_selection import train_test_split

/home/zahradnik/PycharmProjects/master_thesis/src/modules/__init__.py
/home/zahradnik/PycharmProjects/master_thesis


In [7]:
%load_ext autoreload
%autoreload 2

In [58]:
preprocess.preprocessing(        
    athlete_name='zahradnik',
    activity_type='running',
    df_columns = ['timestamp','heart_rate','distance','enhanced_altitude','cadence','enhanced_speed'],
    path_to_load = os.path.join("src","test_activities"),
    name="bechovice"
)

100%|██████████| 1/1 [00:02<00:00,  2.05s/it]


In [75]:
test_df = pd.read_pickle(os.path.join("src","test_activities", "zahradnik","running", "bechovice.pcl")).drop(["enhanced_speed", "cadence", "heart_rate"],axis=1)
cad_cols = [x for x in test_df.columns if "cadence" in x]
test_df.drop(cad_cols,inplace=True,axis=1)
hr_cols = [x for x in test_df.columns if "heart_rate" in x]
test_df.drop(hr_cols,inplace=True,axis=1)

In [63]:
data = fit.load_pcls(
        athlete_name='zahradnik',
        activity_type='running',
        path_to_load=conf["Paths"]["pcl"],
    )

100%|██████████| 386/386 [00:00<00:00, 608.66it/s]


In [64]:
low_dist = []
high_dist = []
for act in data:
    if np.max(act.distance) > 10000:
        high_dist.append(act)
    else:
        low_dist.append(act)
len(low_dist),len(high_dist)

(144, 242)

In [65]:
train_df = fit.clean_data(pd.concat(high_dist))
len(train_df)

681291

In [66]:
speed_train = train_df.enhanced_speed
heart_train = train_df.heart_rate
cad_train = train_df.cadence

train_df.drop("enhanced_speed", axis=1, inplace=True)
train_df.drop("heart_rate", axis=1, inplace=True)
train_df.drop("cadence", axis=1, inplace=True)

# Grid search - cad

In [67]:
from xgboost import XGBRegressor

In [68]:
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 4],
    'min_child_weight': [1, 3]
}

In [69]:
from sklearn.model_selection import GridSearchCV

# Create an XGBoostRegressor instance
clf = XGBRegressor()

# Instantiate GridSearchCV with the XGBoostRegressor, parameter grid, and cross-validation strategy
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=2, scoring='neg_root_mean_squared_error', n_jobs=-1)

# Perform Grid Search on the training data
grid_search.fit(train_df[test_df.columns], cad_train)

# Get the best parameters and best estimator from Grid Search
best_params = grid_search.best_params_
best_clf = grid_search.best_estimator_

# Print the best parameters found by Grid Search
print("Best Parameters: ", best_params)

train_df['cadence'] = cad_train

Best Parameters:  {'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100}


In [78]:
test_df["cadence"] = 1
test_df= preprocess.calc_windows(df=test_df,
          lagged=15,
          cols=["cadence"])
test_df = preprocess.calc_moving(df=test_df, max_range=110, col="cadence")

# grid search - heart

In [79]:
# Create an XGBoostRegressor instance
clf = XGBRegressor()

# Instantiate GridSearchCV with the XGBoostRegressor, parameter grid, and cross-validation strategy
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=2, scoring='neg_root_mean_squared_error', n_jobs=-1)

# Perform Grid Search on the training data
grid_search.fit(train_df[test_df.columns], heart_train)

# Get the best parameters and best estimator from Grid Search
best_params = grid_search.best_params_
best_clf = grid_search.best_estimator_

# Print the best parameters found by Grid Search
print("Best Parameters: ", best_params)

train_df['heart_rate'] = heart_train

Best Parameters:  {'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100}


In [81]:
test_df['heart_rate'] = 1
test_df= preprocess.calc_windows(df=test_df,
          lagged=15,
          cols=["heart_rate"])
test_df = preprocess.calc_moving(df=test_df, max_range=110, col="heart_rate")

# grid search - speed

In [82]:

# Create an XGBoostRegressor instance
clf = XGBRegressor()

# Instantiate GridSearchCV with the XGBoostRegressor, parameter grid, and cross-validation strategy
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=2, scoring='neg_root_mean_squared_error', n_jobs=-1)

# Perform Grid Search on the training data
grid_search.fit(train_df[test_df.columns], speed_train)

# Get the best parameters and best estimator from Grid Search
best_params = grid_search.best_params_
best_clf = grid_search.best_estimator_

# Print the best parameters found by Grid Search
print("Best Parameters: ", best_params)

# Predict using the best estimator
y_pred_cad = best_clf.predict(test_df[test_df.columns])

Best Parameters:  {'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 3, 'n_estimators': 100}


# TPE - cad

In [16]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def objective(trial):
    # Define the hyperparameter search space
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
    }

    # Create an XGBoostRegressor with the current hyperparameters
    xgb_reg = XGBRegressor(**params, random_state=42)

    # Fit the model on the training data
    xgb_reg.fit(X_train, y_train)

    # Make predictions on the validation set
    y_pred = xgb_reg.predict(X_valid)

    # Calculate the mean squared error (you can use other metrics as well)
    mse = mean_squared_error(y_valid, y_pred)

    return mse

In [17]:
# cad
X = train_df
y = cad_train

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a study object and optimize the objective function using TPE
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1)  # You can adjust the number of trials

# Print the best hyperparameters and best objective value
best_params = study.best_params
best_mse = study.best_value

print(f"Best Hyperparameters: {best_params}")
print(f"Best Mean Squared Error: {best_mse}")

train_df['cadence'] = cad_train
test_df= preprocess.calc_windows(df=test_df,
          lagged=15,
          cols=["cadence"])
test_df = preprocess.calc_moving(df=test_df, max_range=110, col="cadence")

[I 2023-10-22 20:58:15,190] A new study created in memory with name: no-name-2c7c10d7-9943-4db7-885d-81b3987c6a72
[I 2023-10-22 21:02:57,430] Trial 0 finished with value: 0.1966899770924631 and parameters: {'n_estimators': 786, 'max_depth': 9, 'learning_rate': 0.0037793030819395857, 'subsample': 0.6617899553347325, 'colsample_bytree': 0.9997721073176253}. Best is trial 0 with value: 0.1966899770924631.


Best Hyperparameters: {'n_estimators': 786, 'max_depth': 9, 'learning_rate': 0.0037793030819395857, 'subsample': 0.6617899553347325, 'colsample_bytree': 0.9997721073176253}
Best Mean Squared Error: 0.1966899770924631


In [18]:
# HR
X = train_df
y = heart_train

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a study object and optimize the objective function using TPE
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1)  # You can adjust the number of trials

# Print the best hyperparameters and best objective value
best_params = study.best_params
best_mse = study.best_value

print(f"Best Hyperparameters: {best_params}")
print(f"Best Mean Squared Error: {best_mse}")
train_df['heart_rate'] = cad_train
test_df= preprocess.calc_windows(df=test_df,
          lagged=15,
          cols=["heart_rate"])
test_df = preprocess.calc_moving(df=test_df, max_range=110, col="heart_rate")

[I 2023-10-22 21:02:59,457] A new study created in memory with name: no-name-2733d086-273a-4fa1-acde-9e9cf39e88d6
[I 2023-10-22 21:06:39,814] Trial 0 finished with value: 22.056738793793166 and parameters: {'n_estimators': 933, 'max_depth': 5, 'learning_rate': 0.0014819093630162221, 'subsample': 0.9916603544787748, 'colsample_bytree': 0.6536177660141926}. Best is trial 0 with value: 22.056738793793166.


Best Hyperparameters: {'n_estimators': 933, 'max_depth': 5, 'learning_rate': 0.0014819093630162221, 'subsample': 0.9916603544787748, 'colsample_bytree': 0.6536177660141926}
Best Mean Squared Error: 22.056738793793166


In [19]:
# speed
X = train_df
y = speed_train

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a study object and optimize the objective function using TPE
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1)  # You can adjust the number of trials

# Print the best hyperparameters and best objective value
best_params = study.best_params
best_mse = study.best_value

print(f"Best Hyperparameters: {best_params}")
print(f"Best Mean Squared Error: {best_mse}")


[I 2023-10-22 21:06:42,514] A new study created in memory with name: no-name-a68bdb9f-0506-43a2-8f6e-2ea59ba23851
[I 2023-10-22 21:07:43,037] Trial 0 finished with value: 3.220358235676995 and parameters: {'n_estimators': 437, 'max_depth': 3, 'learning_rate': 0.0014680073509838298, 'subsample': 0.8208225253868622, 'colsample_bytree': 0.758230896331077}. Best is trial 0 with value: 3.220358235676995.


Best Hyperparameters: {'n_estimators': 437, 'max_depth': 3, 'learning_rate': 0.0014680073509838298, 'subsample': 0.8208225253868622, 'colsample_bytree': 0.758230896331077}
Best Mean Squared Error: 3.220358235676995


# Random search

In [21]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# Load your training data (X_train, y_train) and test data (X_test, y_test) here

# Define the XGBoostRegressor model
xgb_regressor = xgb.XGBRegressor()
  
# Define the hyperparameters and their ranges
param_dist = {
    'learning_rate': uniform(0.001, 0.1),  # Continuous uniform distribution between 0.01 and 0.3
    'n_estimators': randint(10, 1000),  # Discrete uniform distribution between 50 and 200
    'max_depth': randint(3, 10),  # Discrete uniform distribution between 1 and 10
    'subsample': uniform(0.6, 1.0),  # Continuous uniform distribution between 0.5 and 1
    'colsample_bytree': uniform(0.6, 1.0),  # Continuous uniform distribution between 0.5 and 1
}

# Perform RandomizedSearchCV with 5-fold cross-validation
random_search = RandomizedSearchCV(
    estimator=xgb_regressor,
    param_distributions=param_dist,
    n_iter=10,  # Number of parameter settings that are sampled
    cv=5,  # 5-fold cross-validation
    n_jobs=-1,  # Use all available CPUs
    random_state=42
)

# Fit the random search to the training data
random_search.fit(X_train, y_train)

# Print the best hyperparameters found by RandomizedSearchCV
print("Best Hyperparameters: ", random_search.best_params_)

# Evaluate the model with the best hyperparameters on the test data
best_xgb_regressor = random_search.best_estimator_
mse = mean_squared_error(y_valid, best_xgb_regressor.predict(X_valid))
print("Mean Squared Error with Best Hyperparameters: ", mse)


Best Hyperparameters:  {'colsample_bytree': 0.7933755828319241, 'gamma': 0.965255307264138, 'learning_rate': 0.1921102743060054, 'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 138, 'subsample': 0.9934434683002586}
Mean Squared Error with Best Hyperparameters:  0.29000487587217594


In [24]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np

# Load your features (X) and target variable (y) here
# X, y = ...

# Define the objective function to minimize (mean squared error)
def objective(params):
    # Convert float hyperparameters to integer where necessary
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    
    # Create XGBRegressor with given hyperparameters
    xgboost_regressor = xgb.XGBRegressor(**params)
    
    # Use cross-validation to estimate performance
    mse = -np.mean(cross_val_score(xgboost_regressor, X, y, cv=2, scoring='neg_mean_squared_error'))
    return mse

# Define the hyperparameter search space
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 1),  # Discrete uniform distribution
    'max_depth': hp.quniform('max_depth', 3, 10, 1),            # Discrete uniform distribution
    'learning_rate': hp.loguniform('learning_rate', -5, 0),      # Log uniform distribution
    'subsample': hp.uniform('subsample', 0.1, 1),                # Uniform distribution
    'gamma': hp.uniform('gamma', 0, 1)                           # Uniform distribution
}

# Perform Bayesian Optimization
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)  # Number of evaluations

# Print the best hyperparameters found by Bayesian Optimization
print("Best Hyperparameters: ", best)

# Train the final XGBRegressor with the best hyperparameters
best_xgboost_regressor = xgb.XGBRegressor(n_estimators=int(best['n_estimators']),
                                          max_depth=int(best['max_depth']),
                                          learning_rate=best['learning_rate'],
                                          subsample=best['subsample'],
                                          gamma=best['gamma'])

best_xgboost_regressor.fit(X, y)

# Now you can use this best_xgboost_regressor for predictions.


100%|██████████| 10/10 [07:07<00:00, 42.79s/trial, best loss: 1.2913439365000634]
Best Hyperparameters:  {'gamma': 0.0735050709385231, 'learning_rate': 0.024401343807493238, 'max_depth': 6.0, 'n_estimators': 186.0, 'subsample': 0.8335996774702147}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0735050709385231, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.024401343807493238,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=186, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)